In [1]:
## Imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window, to_timestamp, count, countDistinct, desc, split
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, ArrayType

In [2]:
spark = SparkSession \
        .builder \
        .getOrCreate()
# .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0") \

spark.sparkContext.setLogLevel("ERROR")

21/12/15 20:46:58 WARN Utils: Your hostname, cevat resolves to a loopback address: 127.0.1.1; using 192.168.1.152 instead (on interface enp2s0)
21/12/15 20:46:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/15 20:47:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/15 20:47:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/15 20:47:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
schema = StructType([
    StructField("user_id",IntegerType(), False),
    StructField("status",StringType(), False),
    StructField("film_watch",StringType(), False),
    StructField("movie", ArrayType(StructType([
      StructField("show_id",IntegerType(),True),
      StructField("title", StringType(), True),
      StructField("listed_in", StringType(), True)
    ])), True),
    StructField("timestamp",StringType(), False),
])

In [4]:
df = spark.read.json('write_activity_to_kafka/aa.json').filter(col('movie').isNotNull()).withColumn('timestamp', to_timestamp(col('timestamp'), 'yyyy-L-d H:m:s'))
df.printSchema()

AnalysisException: cannot resolve 'movie' given input columns: [];
'Filter isnotnull('movie)
+- Relation [] json


In [16]:
## online sayısı
online_sayisi = len(df.groupBy('user_id').count().collect())
print(online_sayisi)

25


In [ ]:
## Canlı izlenilen filmler
df.groupBy('movie.title').agg(countDistinct('user_id').alias('movie_count')).orderBy(desc('movie_count'), desc('title')).show()

In [131]:
## Category boys
# flatMap(lambda x: str(x).split(',')).map(lambda genre: (genre, 1)).reduceByKey(lambda a,b: a + b).collect()

df.groupBy('user_id', 'movie.title', 'movie.listed_in').count().select('listed_in').rdd.flatMap(lambda x: x[0].split(',')).map(lambda x: (x, 1)).reduceByKey(lambda x, b: x + b).toDF(['genre', 'count']).orderBy(desc('count')).show()

+--------------------+-----+
|               genre|count|
+--------------------+-----+
|  Action & Adventure|   34|
| International Mo...|   31|
|              Dramas|   25|
|            Comedies|   24|
|           Thrillers|   17|
|     Romantic Movies|   15|
|            Comedies|   12|
|       Documentaries|   11|
|      Anime Features|   11|
|Children & Family...|   10|
|              Dramas|    8|
|           Thrillers|    6|
|       Sports Movies|    6|
|    Sci-Fi & Fantasy|    5|
|       Horror Movies|    4|
|    Music & Musicals|    4|
|         Cult Movies|    3|
|      Classic Movies|    3|
|      Classic Movies|    2|
| Children & Famil...|    2|
+--------------------+-----+
only showing top 20 rows

